In [194]:
import pandas as pd
import numpy as np

In [195]:
train = pd.read_csv("train.csv")
test = pd.read_csv("val.csv")
unlabeled = pd.read_csv("unlabeled.csv")

In [196]:
train = train.replace('na', np.NaN)
test = test.replace('na', np.NaN)
unlabeled = unlabeled.replace('na', np.NaN)

In [197]:
train = train.sample(frac=1)

In [198]:
test = test.sample(frac=1)

In [199]:
unlabeled = unlabeled.sample(frac=1)

In [200]:
for x in train.columns[train.dtypes=='object']: train[x] = train[x].astype('float')
for x in test.columns[test.dtypes=='object']: test[x] = test[x].astype('float')
for x in unlabeled.columns[unlabeled.dtypes=='object']: unlabeled[x] = unlabeled[x].astype('float')

In [201]:
true = test['class']

In [202]:
y_train = train["class"]

In [203]:
train = train.drop(['objid', 'class', 'rowc', 'colc', 'ra', 'dec'], axis=1)

In [204]:
test = test.drop(['objid', 'class', 'rowc', 'colc', 'ra', 'dec' ], axis=1)

In [205]:
unlabeled = unlabeled.drop(['objid', 'rowc', 'colc', 'ra', 'dec'], axis=1)

In [206]:
print(train.shape)
print(test.shape)
print(unlabeled.shape)

(30000, 38)
(23333, 38)
(23333, 38)


In [207]:
y_train.shape

(30000,)

In [208]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [109]:
#impute
data = np.asarray(train)
y = np.asarray(y_train)
unlabeled1 = np.asarray(unlabeled)
from sklearn.preprocessing import Imputer
imp = Imputer()
imp.fit(data)
data = imp.transform(data)
data_test = imp.transform(test)
unlabeled1 = imp.transform(unlabeled1)
scaler.fit(data)
data = scaler.transform(data)
data_test = scaler.transform(data_test)
unlabeled1 = scaler.transform(unlabeled1)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [178]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

parameters = {
    'objective': 'multiclass',
    'num_threads': 4,
    'num_class': 3
}

n_rounds = 1000

RANDOM_SEED = 45
clf = LGBMClassifier(n_estimators=n_rounds, **parameters1)
clf2 = LGBMClassifier(n_estimators=n_rounds, **parameters2)
clf3 = LGBMClassifier(n_estimators=n_rounds, **parameters)
lr = LogisticRegression()
np.random.seed(RANDOM_SEED)
sclf = StackingClassifier(classifiers=[clf, clf2, clf3], meta_classifier=lr)
pred = sclf.fit(train, y_train).predict(unlabeled)

In [117]:
unlabeled2 = imp.transform(unlabeled)

In [118]:
unlabeled2 = scaler.transform(unlabeled2)

In [180]:
train.shape

(30000, 38)

In [119]:
data=np.concatenate((data, unlabeled2))

In [179]:
data.shape

(76666, 38)

In [121]:
data_test.shape

(23333, 38)

In [184]:
y_train1 = np.asarray(y_train)

In [185]:
y1 = np.concatenate((y_train1, pred))

In [181]:
data=np.concatenate((train, unlabeled))

In [182]:
data.shape

(53333, 38)

In [183]:
np.isnan(data).any()

True

In [124]:

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier


In [147]:
parameters1 = {
    'objective': 'multiclass',
    'num_threads': 4,
    'num_class': 3,
    'learning_rate': 0.05,
    'bagging_fraction': 0.6,
    'bagging_freq': 100,
    'feature_fraction': 0.9
}

In [126]:
xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.8,
    'silent': 1,
    'subsample': 0.5,
    'objective': 'multi:softmax',
    'max_depth': 10,
    'nrounds': 100,
    'num_class': 3
}
from xgboost import XGBClassifier

In [127]:
parameters2 = {
    'objective': 'multiclass',
    'num_threads': 4,
    'num_class': 3,
    'learning_rate': 0.13,
    'bagging_fraction': 0.7,
    'bagging_freq': 1000,
    'feature_fraction': 0.8
}

In [85]:
from keras import *

Using TensorFlow backend.


In [86]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [90]:
from keras.wrappers.scikit_learn import KerasClassifier

In [210]:
RANDOM_SEED = 44
clf = XGBClassifier(n_estimators=n_rounds, **xgb_params)
clf2 = LGBMClassifier(n_estimators=n_rounds, **parameters2)
clf3 = LGBMClassifier(n_estimators=n_rounds, **parameters)

lr = LogisticRegression()
np.random.seed(RANDOM_SEED)
sclf = StackingClassifier(classifiers=[clf, clf2, clf3], meta_classifier=lr)



In [192]:
y1.shape

(53333,)

In [191]:
data.shape

(53333, 38)

In [211]:
test.shape

(23333, 38)

In [ ]:
yhat = sclf.fit(train, y_train).predict(test)
  

array([0, 0, 0, ..., 0, 0, 0])

In [190]:
f1_score(true, yhat, average="macro")

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1140: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.2424225895016273